In [1]:
import os
import cv2
import numpy as np
from tqdm import tqdm
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping


In [ ]:
def extract_histogram_features_fixed_resize(image_path, resize_shape=(200, 200), gray_levels=8, grid_size=(10, 10)):
    # Load image in grayscale
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    
    # Resize to fixed size (e.g., 200x200)
    img = cv2.resize(img, resize_shape)

    # Calculate size of each patch
    patch_h = resize_shape[0] // grid_size[0]
    patch_w = resize_shape[1] // grid_size[1]

    features = []

    # Calculate gradients using Sobel
    grad_x = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=3)
    grad_y = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=3)
    gradient_magnitude = cv2.magnitude(grad_x, grad_y)

    # Bin sizes
    gray_bin_size = 256 // gray_levels
    grad_max = gradient_magnitude.max()
    grad_bin_size = grad_max / gray_levels if grad_max != 0 else 1

    for i in range(grid_size[0]):
        for j in range(grid_size[1]):
            y1, y2 = i * patch_h, (i + 1) * patch_h
            x1, x2 = j * patch_w, (j + 1) * patch_w

            patch = img[y1:y2, x1:x2]
            grad_patch = gradient_magnitude[y1:y2, x1:x2]

            # Histogram of intensity
            hist_intensity, _ = np.histogram(patch, bins=gray_levels, range=(0, 256))
            hist_intensity = hist_intensity.astype(np.float32) / (patch.size + 1e-6)

            # Histogram of gradients
            hist_grad, _ = np.histogram(grad_patch, bins=gray_levels, range=(0, grad_max))
            hist_grad = hist_grad.astype(np.float32) / (grad_patch.size + 1e-6)

            # Combine
            patch_features = np.concatenate([hist_intensity, hist_grad])
            features.append(patch_features)

    # Final 1600-dimensional vector
    return np.concatenate(features)



In [ ]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from tensorflow.keras.utils import to_categorical

# فرض: X = لیست بردارهای ویژگی (n_samples, 1600)
#      y = لیست لیبل‌های متنی ('Tumor', 'Cyst', ...)

X = np.array(X)  # شکل: (n_samples, 1600)
y = np.array(y)  # شکل: (n_samples,)

# نرمال‌سازی ویژگی‌ها بین 0 تا 1
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# reshape به فرم توالی (sequence)
X_seq = X_scaled.reshape(-1, 100, 16)  # مناسب برای LSTM

# تبدیل لیبل به عدد + one-hot
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
y_onehot = to_categorical(y_encoded, num_classes=4)


[[3.125e-02 0.000e+00 0.000e+00 ... 1.500e-04 4.000e-04 3.500e-04]
 [3.125e-02 0.000e+00 0.000e+00 ... 1.000e-04 2.000e-04 1.500e-04]
 [3.125e-02 0.000e+00 0.000e+00 ... 6.000e-04 9.000e-04 6.500e-04]
 ...
 [3.125e-02 0.000e+00 0.000e+00 ... 1.000e-04 1.500e-04 0.000e+00]
 [3.125e-02 0.000e+00 0.000e+00 ... 5.000e-05 1.000e-04 1.000e-04]
 [3.125e-02 0.000e+00 0.000e+00 ... 5.000e-05 0.000e+00 1.500e-04]] [0 0 0 ... 3 3 3]


In [33]:
model = build_lstm_model()

history = model.fit(
    X_seq, y_onehot,
    epochs=50,
    batch_size=32,
    validation_split=0.2,
    verbose=1
)


Epoch 1/50
 15/312 ━━━━━━━━━━━━━━━━━━━━ 36s 123ms/step - accuracy: 0.4638 - loss: 1.1907

KeyboardInterrupt: 

In [ ]:
# Assuming y_test is one-hot encoded like y_onehot
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=1)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")


InvalidArgumentError: Graph execution error:

Detected at node StatefulPartitionedCall/sequential_8_1/lstm_8_1/while/body/_15/sequential_8_1/lstm_8_1/while/lstm_cell_1/MatMul defined at (most recent call last):
<stack traces unavailable>
Matrix size-incompatible: In[0]: [32,1], In[1]: [16,800]
	 [[{{node StatefulPartitionedCall/sequential_8_1/lstm_8_1/while/body/_15/sequential_8_1/lstm_8_1/while/lstm_cell_1/MatMul}}]] [Op:__inference_multi_step_on_iterator_436656]